In [19]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
%matplotlib inline

In [38]:
# Возьмем 2 признака и 1000 объектов
n_features = 2
n_objects = 1000

# сгенерируем вектор истинных весов
w_true = np.random.normal(size=(1, n_features ))

# сгенерируем матрицу X, вычислим Y с добавлением случайного шума
X = np.random.uniform(-7, 7, (n_objects, n_features))
shift=np.random.uniform(-3, 3)
Y = X.dot(w_true.T).flatten()+shift + np.random.normal(0, 0.5, size=(n_objects,))

#Рассчитываем вероятности
Prob_negative=norm.pdf(Y, loc=3, scale=1.5)
Prob_positive=norm.pdf(Y, loc=8, scale=0.9)
prob=Prob_negative/(Prob_negative+Prob_positive)

#Генерируем лейблы классов
Y = np.where(np.random.uniform(size=Y.size)>prob, 1, 0)

# разбивка на обучающую и тестовую выборки
train_proportion = 0.7
train_test_cut = int(len(X) * train_proportion)

shuffle_index = np.random.permutation(len(X))
X_shuffled, Y_shuffled = X[shuffle_index], Y[shuffle_index]

X_train, X_test, y_train, y_test = \
    X_shuffled[:train_test_cut], \
    X_shuffled[train_test_cut:], \
    Y_shuffled[:train_test_cut], \
    Y_shuffled[train_test_cut:]

In [39]:
def confusion(predicted, actual):
    TP=np.sum(np.logical_and(predicted==1, actual==1))
    FP=np.sum(np.logical_and(predicted==1, actual==0))
    FN=np.sum(np.logical_and(predicted==0, actual==1))
    TN=np.sum(np.logical_and(predicted==0, actual==0))
    return {'TP': TP, 'FP': FP, 'FN': FN, 'TN': TN}

In [40]:
class logistic_regression:
    def __init__(self, n_iterations=1000, eta=0.05):
        self.n_iterations=1000
        self.eta=eta
    @staticmethod
    def log_grad(w, c, X, target):
        m = X.shape[0]
        y=(2*target-1)
        score=np.dot(X, w.T).flatten()
        Z=-y/(m*(1+np.exp(y*score)))
        grad=Z[np.newaxis, :].dot(X)
        return grad/m, np.sum(Z)/m
    @classmethod
    def optimize(cls, w, c, X, y, n_iterations, eta):
        for i in range(n_iterations):        
            grad_w, grad_c = cls.log_grad(w, c, X, y)
            w = w - eta * grad_w
            c = c - eta * grad_c
        return w, c
    def fit(self, X, y):
        w0 = np.zeros((1, X.shape[1]))
        c0=0
        self.w, self.c= self.optimize(w0, c0, X, y, self.n_iterations, self.eta)
    def predict_proba(self, X):  
        '''
        Рассчёт вероятности
        '''
        score=X.dot(self.w.T).flatten()+self.c
        return 1/(1+np.exp(-score))
    def predict(self, X, thr=0.5):
        proba=self.predict_proba(X)
        y_predicted = np.zeros(proba.shape, dtype=bool) 
        y_predicted[proba>thr]=1
        y_predicted[proba<=thr]=0
        return y_predicted

In [41]:
mod=logistic_regression()

mod.fit(X_train, y_train)
y_predicted_test = mod.predict(X_test)
confusion(y_predicted_test, y_test)

num_pos=np.sum(y_test)
num_neg=len(y_test)-num_pos
print(f'{num_pos} объектов положительного класса, {num_neg} объектов негативного класса')
print()

#Чувствительность
TPR=np.concatenate([[0], np.cumsum(1-y_test)/num_neg])

#Ложно-позитивный уровень
FPR=np.concatenate([[0], np.cumsum(y_test)/num_pos])

AUC_ROC = np.sum(0.5*(FPR[1:]-FPR[:-1])*(TPR[1:]+TPR[:-1]))

precision=np.cumsum(1-y_test)/(np.cumsum(1-y_test)+np.cumsum(y_test))
precision

recall = TPR

AUC_PR = np.sum(precision*(recall[1:]-recall[:-1]))

print(f'AUC_ROC = {AUC_ROC:.3f}, AUC_PR = {AUC_PR:.3f}')


62 объектов положительного класса, 238 объектов негативного класса

AUC_ROC = 0.544, AUC_PR = 0.826
